# 4. Evaluation

## 4.1. О качестве моделей

Довольно простые модели вроде линейной показывают довольно хорошую точность по метрике F1 Macro. Единственный атрибут, с предсказанием которого есть проблемы - bedding, там значение метрики не превышает 0.65, когда при предсказании других полей получается 0.88 и более у лучшей модели. К сожалению, я уже не успеваю разобраться, что именно идёт не так с этим полем. Учитывая модульность решения, в будущем можно было бы заменить модель только для этого признака.

Две модели, вышедшие вперёд по качеству - RidgeClassifier и LinearSVC. Для выбора той, что пойдёт в итоговое решение, нужно проверить, соответствуют ли они требованию по скорости работы. 

## 4.2. О скорости работы моделей

Давайте попробуем оценить пропускную способность моделей и прогоним их на всей исходной выборке rates_dirty, замеряя время:   

In [1]:
import pandas as pd

rates_dirty = pd.read_csv("../raw_data/rates_dirty.csv")
number_of_records = len(rates_dirty)

In [6]:
import time
import os

start = time.time()
os.system("python3 ../cli_app.py --content ../raw_data/rates_dirty.csv --model svc_pipeline > /dev/null")
end = time.time()

In [7]:
print(f"Elapsed time: {end - start:.2f} s")
print(f"SVC throughput: {number_of_records / (end - start):.2f} records per second")

Elapsed time: 6.93 s
SVC throughput: 26024.08 records per second


In [8]:
import time
import os

start = time.time()
os.system("python3 ../cli_app.py --content ../raw_data/rates_dirty.csv --model ridge_pipeline > /dev/null")
end = time.time()

In [11]:
print(f"Elapsed time: {end - start:.2f} s")
print(f"Ridge throughput: {number_of_records / (end - start):.2f} records per second")

Elapsed time: 7.03 s
Ridge throughput: 25684.03 records per second


Посмотрим, сколько занимает загрузка модели. Попробуем протестировать предсказания по вне скрипта с уже загруженными моделями:

In [10]:
from model.manager import ModelManager

ridge_manager = ModelManager("ridge_pipeline")
print(ridge_manager.class_to_model.keys())

svc_manager = ModelManager("svc_pipeline")
print(svc_manager.class_to_model.keys())

dict_keys(['class', 'quality', 'bathroom', 'bedding', 'capacity', 'club', 'balcony', 'view'])
dict_keys(['class', 'quality', 'bathroom', 'bedding', 'capacity', 'club', 'balcony', 'view'])


In [12]:
without_na = rates_dirty.fillna("undefined")

start = time.time()
svc_manager.predict(without_na)
end = time.time()

In [13]:
print(f"Elapsed time: {end - start:.2f} s")
print(f"SVC throughput: {number_of_records / (end - start):.2f} records per second")

Elapsed time: 5.15 s
SVC throughput: 35051.44 records per second


In [14]:
start = time.time()
ridge_manager.predict(without_na)
end = time.time()

In [15]:
print(f"Elapsed time: {end - start:.2f} s")
print(f"Ridge throughput: {number_of_records / (end - start):.2f} records per second")

Elapsed time: 5.28 s
Ridge throughput: 34196.71 records per second


Как можно видеть по результатам выше, обе модели справляются с необходимой пропускной способностью в 4000 записи в секунду. Это значит, что мы можем выбрать более качественную модель - LinearSVC - в качестве финальной 